## Modelo Vetorial

### Importando-se os pacotes necessários para a atividade

In [1]:
import pandas as pd
import re
import operator
from math import log
from collections import Counter
from unicodedata import normalize

### Obtendo-se a coleção

In [2]:
data_frame = pd.read_csv("results.csv")
documentos = data_frame["text"].tolist()

### 1) Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice

#### Gerando-se os índices invertidos

#### Algoritmo "parse" para obter os tokens de um determinado documento

In [3]:
def is_pontuacao(token):
    '''
        Retorna-se um booleano indicando se o token passado por parâmetro é uma pontuação.
    '''
    lista_pontuacoes = [",", ".", "!", "?", ":", "/", "#", "*", "(", ")", ";", ""]
    return token.strip() in lista_pontuacoes

def remove_acentuacoes(token):
    '''
        Remove as acentuações de um token.
    '''
    token_unicode = unicode(token, "utf-8")
    return normalize('NFKD', token_unicode).encode('ASCII','ignore')

def parse(texto):
    '''
        Transforma o texto em uma lista de tokens, eliminando espaços vazios, acentuações e pontuações, e
        tranforma todas as palavras em letras minúsculas.
    '''
    texto = re.sub(r'[,.!?:*();]', ' ', str(texto))
    texto = re.sub(r'\n', ' ', texto)
    lista_texto = texto.split(' ')

    tokens = []
    for palavra in lista_texto:
        token = palavra.lower()
        token = token.strip()
        token = remove_acentuacoes(token)

        if(not is_pontuacao(token)):
            tokens.append(token)
    
    return tokens

#### Algoritmo para obtenção dos índices invertidos com frequência (IDF)

In [4]:
def get_indices_invertidos_com_frequencia(documentos):
    '''
        Obtém uma lista de índices invertidos a partir de uma lista de documentos,
        com a frequência do índice em cada um desses documentos.
    '''
    indices_invertidos = {}
    for i in range(len(documentos)):
        doc = parse(documentos[i])
        documento_counter = Counter(doc)
        
        for indice in documento_counter.keys():
            if(len(indice) > 3):
                if(indice in indices_invertidos):
                    indices_invertidos[indice][i] = documento_counter[indice]
                else:
                    indices_invertidos[indice] = { i: documento_counter[indice] }
    
    return indices_invertidos

### 2) Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário.

#### Obtendo-se os índices com IDF

In [5]:
indices_invertidos_com_frequencia = get_indices_invertidos_com_frequencia(documentos)

### 3) Implementar as seguintes versões do modelo vetorial:

    1. Representação binária;
    2. TF (lembre-se que esse modelo já está implementado);
    3. TF-IDF;
    4. BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

#### Algoritmos úteis para as implementações

In [6]:
def get_tuplas_documentos_ordenados_peso(dic_documentos):
    '''
        Obtém-se tuplas de documentos ordenados decrescentemente pelo peso, a partir de um dicionário
        que tem como chave o documento, e valor o peso.
    '''
    documentos_tuplas = dic_documentos.items()
    indice_peso = 1
    documentos_tuplas.sort(key = operator.itemgetter(indice_peso), reverse=True)
    
    return documentos_tuplas

def get_K_primeiros_ids_documentos(documentos_tuplas, k):
    '''
        Obtendo-se a lista de id dos k primeiros documentos, a partir de uma lista de tuplas, as quais
        o primeiro índice é o id do documento e o segundo é o seu peso.
    '''
    id_documentos_ordenados = [id_documento for id_documento, peso in documentos_tuplas]
    k_primeiros_id_documentos = id_documentos_ordenados[:k]
    
    return k_primeiros_id_documentos

def idf(quantidade_documentos_colecao, quantidade_documentos_com_palavra):
    '''
        Calcula o Inverse Document Frequency (IDF).
    '''
    return log((quantidade_documentos_colecao + 1) / quantidade_documentos_com_palavra)

#### Implementação do algoritmo para a *Representação Binária*

In [7]:
def get_documentos_ordem_representacao_binaria(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Representação Binária",
        na qual, os documentos que possuem a maior quantidade de termos da consultas
        são os mais representativos.
    '''
    consulta = consulta.split(" ")
    # Separando-se os termos da pesquisa, de forma a evitar a contagem de palavras repetidas. Por exemplo,
    # uma consulta por "thaynan andrey thaynan", evitaria que eu calculasse "thaynan" duas vezes.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            for id_doc in indices_invertidos[termo].keys():
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + 1
                else:
                    documentos_com_peso[id_doc] = 1
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *TF*

In [8]:
def get_documentos_ordem_TF(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Term Frequency Weighting",
        na qual, quanto mais o termo se repete na consulta ou no documento, mais pesos
        ele tem.
    '''
    consulta = consulta.split(" ")
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            
            for id_doc in indices_invertidos[termo].keys():
                peso_doc_termo = indices_invertidos[termo][id_doc]
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *TF-IDF*

In [9]:
def get_documentos_ordem_TF_IDF(consulta, indices_invertidos, k_primeiros_elementos):
    '''
        Obtém os documentos ordenados segundo a ideia de "Term Frequency Weighting" acrescentado do IDF,
        no qual, termos muito populares no documento são penalizados e termos pouco populares são valorizados,
        o que evita que stop words se sobreponham a outras palavras.
    '''
    consulta = consulta.split(" ")
    quantidade_documentos_colecao = len(documentos)
    
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            quantidade_documentos_com_termo = len(indices_invertidos[termo].keys())
            
            for id_doc in indices_invertidos[termo].keys():
                peso_doc_termo = indices_invertidos[termo][id_doc] * idf(quantidade_documentos_colecao, quantidade_documentos_com_termo)
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k_primeiros_elementos)
    
    return k_primeiros_id_documentos

#### Implementação do algoritmo para a *BM25*

In [10]:
def get_documentos_ordem_bm25(consulta, indices_invertidos, k):
    '''
        Obtém os documentos ordenados segundo a ideia do BM25.
    '''
    consulta = consulta.split(" ")
    quantidade_documentos_colecao = len(documentos)
    
    # Separando-se os termos da pesquisa, de forma a contar a repetição de cada termo,
    # para assim utilizá-lo no cálculo do peso.
    termos_counter = Counter(consulta)
    
    # Obtendo-se os documentos e seus respectivos pesos para a consulta
    documentos_com_peso = {}
    for termo in termos_counter.keys():
        
        if(termo in indices_invertidos):
            peso_consulta_termo = termos_counter[termo]
            quantidade_documentos_com_termo = len(indices_invertidos[termo].keys())
            
            for id_doc in indices_invertidos[termo].keys():
                numerador_peso_doc_termo = (k + 1) * indices_invertidos[termo][id_doc] * idf(quantidade_documentos_colecao, quantidade_documentos_com_termo)
                denominador_peso_doc_termo = indices_invertidos[termo][id_doc] + k
                peso_doc_termo = numerador_peso_doc_termo / denominador_peso_doc_termo
                peso_termo = peso_consulta_termo * peso_doc_termo
                
                if(id_doc in documentos_com_peso):
                    documentos_com_peso[id_doc] = documentos_com_peso[id_doc] + peso_termo
                else:
                    documentos_com_peso[id_doc] = peso_termo
    
    documentos_tuplas = get_tuplas_documentos_ordenados_peso(documentos_com_peso)
    k_primeiros_id_documentos = get_K_primeiros_ids_documentos(documentos_tuplas, k)
    
    return k_primeiros_id_documentos

### 4) Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta

### Consultas a serem realizadas

    1. "jair bolsonaro"
    2. "governo do brasil"
    3. "previdencia social"

### Consulta por "jair bolsonaro"

##### Execução do algoritmo *Representação Binária* e apresentação da tabela com o id dos documentos do TOP 5

In [11]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_representacao_binaria("jair bolsonaro", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_representacao_binaria = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_representacao_binaria = tabela_jair_bolsonaro_representacao_binaria.reindex(columns=colunas)

tabela_jair_bolsonaro_representacao_binaria

,Ranking,ID documento
0,1,0
1,2,1
2,3,145
3,4,150
4,5,24


##### Execução do algoritmo *TF* e apresentação da tabela com o id dos documentos do TOP 5

In [12]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF("jair bolsonaro", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF = tabela_jair_bolsonaro_TF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF

,Ranking,ID documento
0,1,150
1,2,206
2,3,165
3,4,18
4,5,41


##### Execução do algoritmo *TF-IDF* e apresentação da tabela com o id dos documentos do TOP 5

In [13]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF_IDF("jair bolsonaro", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF_IDF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF_IDF = tabela_jair_bolsonaro_TF_IDF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF_IDF

,Ranking,ID documento
0,1,206
1,2,150
2,3,165
3,4,18
4,5,215


##### Execução do algoritmo *BM25* e apresentação da tabela com o id dos documentos do TOP 5

In [14]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_bm25("jair bolsonaro", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_BM25 = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_BM25 = tabela_jair_bolsonaro_BM25.reindex(columns=colunas)

tabela_jair_bolsonaro_BM25

,Ranking,ID documento
0,1,206
1,2,150
2,3,165
3,4,215
4,5,236


### Consulta por "governo do brasil"

##### Execução do algoritmo *Representação Binária*

In [15]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_representacao_binaria("governo do brasil", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_representacao_binaria = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_representacao_binaria = tabela_jair_bolsonaro_representacao_binaria.reindex(columns=colunas)

tabela_jair_bolsonaro_representacao_binaria

,Ranking,ID documento
0,1,2
1,2,7
2,3,17
3,4,18
4,5,20


##### Execução do algoritmo *TF*

In [16]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF("governo do brasil", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF = tabela_jair_bolsonaro_TF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF

,Ranking,ID documento
0,1,150
1,2,165
2,3,172
3,4,18
4,5,247


##### Execução do algoritmo *TF-IDF*

In [17]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF_IDF("governo do brasil", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF_IDF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF_IDF = tabela_jair_bolsonaro_TF_IDF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF_IDF

,Ranking,ID documento
0,1,150
1,2,165
2,3,172
3,4,18
4,5,247


##### Execução do algoritmo *BM25*

In [18]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_bm25("governo do brasil", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_BM25 = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_BM25 = tabela_jair_bolsonaro_BM25.reindex(columns=colunas)

tabela_jair_bolsonaro_BM25

,Ranking,ID documento
0,1,165
1,2,172
2,3,247
3,4,18
4,5,114


### Consulta por "previdencia social"

##### Execução do algoritmo *Representação Binária*

In [19]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_representacao_binaria("previdencia social", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_representacao_binaria = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_representacao_binaria = tabela_jair_bolsonaro_representacao_binaria.reindex(columns=colunas)

tabela_jair_bolsonaro_representacao_binaria

,Ranking,ID documento
0,1,139
1,2,35
2,3,165
3,4,172
4,5,212


##### Execução do algoritmo *TF*

In [20]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF("previdencia social", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF = tabela_jair_bolsonaro_TF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF

,Ranking,ID documento
0,1,36
1,2,247
2,3,137
3,4,165
4,5,212


##### Execução do algoritmo *TF-IDF*

In [21]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_TF_IDF("previdencia social", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_TF_IDF = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_TF_IDF = tabela_jair_bolsonaro_TF_IDF.reindex(columns=colunas)

tabela_jair_bolsonaro_TF_IDF

,Ranking,ID documento
0,1,36
1,2,247
2,3,137
3,4,165
4,5,212


##### Execução do algoritmo *BM25*

In [22]:
ranking = range(1, 6)
ids_documentos = get_documentos_ordem_bm25("previdencia social", indices_invertidos_com_frequencia, 5)

tabela_jair_bolsonaro_BM25 = pd.DataFrame({"Ranking": ranking, 
                                 "ID documento": ids_documentos})

# Reordenando as colunas para a apresentação
colunas = ["Ranking", "ID documento"]
tabela_jair_bolsonaro_BM25 = tabela_jair_bolsonaro_BM25.reindex(columns=colunas)

tabela_jair_bolsonaro_BM25

,Ranking,ID documento
0,1,247
1,2,36
2,3,165
3,4,212
4,5,137


### 5) Compare os resultados encontrados e responda.

### 5.1) Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

Fazendo-se uma busca por "previdencial social", espero encontrar documentos que retratem a previdencia social, seja pela situação atual da mesma ou pela reforma, pois o assunto está em foco.  

Nesta perspectiva, observei o top 5 retornado por cada algoritmo, e tornou-se perceptível que o algoritmo por "representação binária" é o que mais se distoa em relação aos resultados, tendo apenas os documentos de identificação "165" e "212" como equivalentes aos demais resultados. Os demais algoritmos retornam o mesmo resultado, tendo a diferença que os algoritmos TF e TF-IDF tem o mesmo ranking e o algoritmo BM25 possui uma ordem diferente para os documentos.  

Para a avaliação, observei o conteúdo de cada artigo para tentar me embasar e ver o quão são uteis para minha busca, fomentando meu próprio ranking dos aritgos. Logo, de todos os resultados, vi que os documentos com id **36**, **247** e **137** falam muito sobre previdência, destacando-se pontos ruins e bons sobre a mesma, tendo-se **destaque o 36** por discutir durante todo o artigo sobre a previdência, contexto históricos e a reforma. Logo, **estão no topo do meu ranking**. Os demais documentos retornados possuem conteúdo sobre a previdência, mas não são o foco principal do tema, pois os mesmos visam mais criticar o governo de Bolsonaro e apresentar problemas do mesmo, dentre eles, eles citam a previdência, mas o foco são apresentar problemas e não discutir sobre a previdência ou sobre a reforma dela. 

Meu Ranking fica:
    1. 36
    2. 247
    3. 137
    4. 139
    5. 165
    
Sendo assim, levando-se em consideração o conteúdo desses artigos e que meu foco nos resultados é falar sobre a previdência, dando total foco a ela, acredito que os artigos **36**, **247** e **137** foram os mais contudentes, por isso estaõ no topo do ranking, o **139** fala bem sobre a previdência, e o **165** fala sobre a mesma, mas a utiliza mais em crítcas. Desses artigos apenas o 139 foi obtido pelo algoritmo de "Representação Binária" o que o desqualifica como o que possui os melhores resultados. Os demais algoritmos retornaram 4 desses documentos, dos quais o TF e o TF-IDF possuem o mesmo top 3 que o meu ranking. O BM25, coloca o **247 a frente do 36**, o que no meu ranking não se configura, ademais, ele considera o **137** menos importante que os documentos **165** e **212**, os quais apesar de falarem da previdência, utilizam-na mais para críticas do que para discutir sobre ela. Logo, se baseando no meu ranking de artigos, acredito que o **TF** e o **TF-IDF** obtiveram os melhores resultados para a minha pesquisa.

### 5.2) Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard).

Utilizando como exemplo a consulta "jair bolsonaro" faço os cálculos par a par entre os algoritmos em busca de obter os que possuem maior similaridade e os que possuem menor. Logo, segue-se abaixo, os cálculos.

#### Algortimos úteis para os cálculos do índice de Jaccard.

In [23]:
def getIntersecao(lista1, lista2):
    '''
        Retorna a interseção de duas listas
    '''
    return list(set(lista1) & set(lista2)) 

def getUniao(lista1, lista2):
    '''
        Retorna a união de duas listas
    '''
    return list(set(lista1) | set(lista2)) 
    
def getIndiceJaccard(conjunto1, conjunto2):
    '''
        Retorna o índice de Jaccard para os dois conjuntos passados.
    '''
    cardinalidade_intersecao_conjuntos = float(len(getIntersecao(conjunto1, conjunto2)))
    cardinalidade_uniao_conjuntos = float(len(getUniao(conjunto1, conjunto2)))
    
    return cardinalidade_intersecao_conjuntos / cardinalidade_uniao_conjuntos

#### Calculando-se os índices de Jaccard

In [24]:
ids_documentos_representcao_binaria = get_documentos_ordem_representacao_binaria("jair bolsonaro", indices_invertidos_com_frequencia, 5)
ids_documentos_tf = get_documentos_ordem_TF("jair bolsonaro", indices_invertidos_com_frequencia, 5)
ids_documentos_tf_idf = get_documentos_ordem_TF_IDF("jair bolsonaro", indices_invertidos_com_frequencia, 5)
ids_documentos_bm25 = get_documentos_ordem_bm25("jair bolsonaro", indices_invertidos_com_frequencia, 5)

# índice de jaccard entre os algoritmos de representação binária e TF
indice_jaccard_rep_binaria_e_tf = getIndiceJaccard(ids_documentos_representcao_binaria, ids_documentos_tf)

# índice de jaccard entre os algoritmos de representação binária e TF-IDF
indice_jaccard_rep_binaria_e_tf_idf = getIndiceJaccard(ids_documentos_representcao_binaria, ids_documentos_tf_idf)

# índice de jaccard entre os algoritmos de representação binária e BM25
indice_jaccard_rep_binaria_e_bm25 = getIndiceJaccard(ids_documentos_representcao_binaria, ids_documentos_bm25)

# índice de jaccard entre os algoritmos de TF e TF-IDF
indice_jaccard_tf_e_tf_idf = getIndiceJaccard(ids_documentos_tf, ids_documentos_tf_idf)

# índice de jaccard entre os algoritmos de TF e BM25
indice_jaccard_tf_e_bm25 = getIndiceJaccard(ids_documentos_tf, ids_documentos_bm25)

# índice de jaccard entre os algoritmos de TF-IDF e BM25
indice_jaccard_tf_idf_e_bm25 = getIndiceJaccard(ids_documentos_tf_idf, ids_documentos_bm25)

#### Tabela para apresentar as similaridades entres os resultados dos algoritmos

In [25]:
algoritmos1 = ["Representação Binária", "Representação Binária", "Representação Binária", "TF", "TF", "TF-IDF"]
algoritmos2 = ["TF", "TF-IDF", "BM25", "TF-IDF", "BM25", "BM25"]
similaridades = [indice_jaccard_rep_binaria_e_tf, indice_jaccard_rep_binaria_e_tf_idf, indice_jaccard_rep_binaria_e_bm25,
                indice_jaccard_tf_e_tf_idf, indice_jaccard_tf_e_bm25, indice_jaccard_tf_idf_e_bm25]

tabela_similaridade = pd.DataFrame({"Algoritmo 1": algoritmos1, "Algoritmo 2": algoritmos2, "Similaridade": similaridades})

# Reordenando as colunas para a apresentação
colunas = ["Algoritmo 1", "Algoritmo 2", "Similaridade"]
tabela_similaridade = tabela_similaridade.reindex(columns=colunas)

tabela_similaridade

,Algoritmo 1,Algoritmo 2,Similaridade
0,Representação Binária,TF,0.111111
1,Representação Binária,TF-IDF,0.111111
2,Representação Binária,BM25,0.111111
3,TF,TF-IDF,0.666667
4,TF,BM25,0.428571
5,TF-IDF,BM25,0.666667


Observa-se que o algoritmo de Representação Binária é o que possui os resultados mais distoantes, sendo, portanto, o que possui a menor similaridade. Enquanto que se apresentam bem similares os resultados dos algoritmos TF com os do TF-IDF, assi como, os resultados do TF-IDF como o BM25. Logo, pode-se ver que o TF-IDF tem um maior indício de similaridade para com os demais algoritmos.